In [1]:
import sqlite3
import pandas as pd

In [2]:
connect=sqlite3.connect('../data/checking-logs.sqlite', check_same_thread=False)

In [3]:
query="""select checker.uid, checker.labname, checker.timestamp as first_commit_ts, pageviews.datetime as first_view_ts
from checker left join pageviews on checker.uid=pageviews.uid
where status='ready' and numTrials = 1 and labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
and checker.uid like 'user_%' and 
(pageviews.datetime=(select min(pageviews.datetime) from pageviews where uid=checker.uid) or pageviews.datetime is null)"""

datamart=pd.io.sql.read_sql(query, connect, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


In [4]:
test=datamart[datamart['first_view_ts'].notnull()]
control=datamart[datamart['first_view_ts'].isnull()]
control=control.fillna(test['first_view_ts'].mean())

In [ ]:
test.to_sql('test', connect)
control.to_sql('control', connect)

In [6]:
connect.close()